In [ ]:
import pandas as pd
import math

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom, poisson, norm
import statsmodels.api as sm

plt.rcParams.update({'font.size': 14})
%config InlineBackend.figure_format = 'retina'


In [ ]:
!wget https://www.wolframcloud.com/objects/282a8450-4101-41cc-aded-2edd3ab69e4d --output-document Prussian-Horse-Kick-Data.csv
!wget https://raw.githubusercontent.com/niquepolice/misc-files/master/StudentData.csv

# Please, enter your full name and telegram username here


# Poisson regression

In [ ]:
# https://datarepository.wolframcloud.com/resources/Prussian-Horse-Kick-Data
# The data give the number of soldiers in the Prussian cavalry killed by horse kicks, by corp membership and by year.

df = pd.read_csv("Prussian-Horse-Kick-Data.csv")
df

In [ ]:
series = df.iloc[:, 1:].values.flatten() 
plt.hist(series, bins=100, label="data")

x = np.arange(5)
plt.scatter(x, poisson.pmf(x, series.mean()) * series.size, color='m', label="poisson")
plt.legend()
plt.show()

# Predicting number of students awards

In [ ]:
#  https://search.r-project.org/CRAN/refmans/gorica/html/academic_awards.html
df = pd.read_csv("StudentData.csv")
df.head()

In [ ]:
df[df.prog == 2].num_awards.hist()

In [ ]:
df_dummy = pd.get_dummies(df, columns=["prog"])
df_dummy

## Simply use statsmodels

In [ ]:
Y = df_dummy["num_awards"].values
X = sm.tools.add_constant(df_dummy[["math", "prog_1", "prog_2", "prog_3"]].values)
fit_res = sm.Poisson(Y, X).fit()
fit_res.summary()


In [ ]:
sm_pred = fit_res.predict(X)

def plot(*preds):
    plt.figure(figsize=(15, 4))
    ax = None
    for prog in [1, 2, 3]:
        ax = plt.subplot(130 + prog, sharey=ax)
        inds = df.prog == prog 
        plt.scatter(df.math[inds], Y[inds], label='y')
        for i, pred in enumerate(preds):
            plt.scatter(df.math[inds], pred[inds], label=f'prediction {i + 1}')
        plt.title(f"prog={prog}")
        plt.legend()
        
plot(sm_pred)

### 1. Exponential family
$$p(y, \lambda) = \frac{e^{-\lambda} \lambda^y}{y!}$$

$$b(y) exp\left(\eta^T T(y) - a(\eta)\right)$$ 
    
<!-- <font color='red'>Show that the Poisson distribution is in the exponential family.</font> -->
Poisson distribution is in the exponential family:
$$\frac{1}{y!} \exp(-y + y \ln \lambda) = \frac{1}{y!} \exp(\eta y - e^\eta)$$

The connection between $\eta$ and $\lambda$ is  $\ln \lambda = \eta$ 

<!-- The connection between $\eta$ and $\lambda$ is:  $\color{red}{?}$  -->

### 2. Decision function 
$$h_\theta(x) = \mathbb E[y | x] = \lambda = e^{\eta} = e^{\theta^\top x}$$

### 3. Objective function / maximize log likelihood 
The loss function for the Poisson regression is
$$f(\theta) = \sum_k \left(e^{x_k^\top \theta} - y_k x_k^\top \theta \right)$$




# Problem 1. (1.5) Gradient

Denoting $X = \begin{pmatrix}x_1^\top \\ \vdots \\ x_n^\top \end{pmatrix}$  the matrix, which rows is our inputs $x_k$ (this is how a bunch of vectors is usually represented in `numpy`), and $Y=\begin{pmatrix}y_1 \\ \vdots \\ y_n \end{pmatrix}$ the vector of outputs, we can rewrite $f$ in the matrix form (so that $f$ could be calculated efficiently with `numpy`'s backend without in-python loops)
$$f(\theta) = \mathbf 1^\top \left(\exp(X\theta) - Y \odot X \theta \right),$$
there $\odot$ denotes componentwise multiplication, and $\exp$ is also elementwise.

- Find the gradient of the loss $\nabla f(\theta)$ and write it **in the matrix form** in the cell below.

# Solution

here

In [ ]:
# and here 
def f(theta: np.ndarray):
    return (np.exp(X @ theta) - Y * (X @ theta)).sum()

    
def gradf(theta: np.ndarray):
    pass

# Problem 2. (1) Gradient Descent
Probably, the simpliest optimization method is the gradient descent with the constant step-size $h$:
$$\theta^{i+1} = \theta^i - h \nabla f(\theta^i).$$

- Implement this algorithm, using the draft below.

- If staring point $\theta^{0}$ is set to $\vec 0$, what is the maximum step-size value $h$, with which the method still could be called a "descent", i.e. $f(\theta^i)$ monotonously decreases? Use f"{h:.0e}" format for the answer.



In [ ]:
dim = X.shape[1] 
theta = np.zeros(dim)

niters = # ?
h = # ?
flog = []
for i in range(niters):
    flog.append(f(theta))
    # antigradient step here
    
    
plt.plot(flog)
plt.title("Loss")
plt.xlabel("iter")


In [ ]:
# maximum h

In [ ]:
def predict(x, theta):
    pass  # decision function here

plot(sm_pred, predict(X, theta))


# Problem 3. (1.5+1) Hessian
- 3.1 (1.5) Find $\nabla^2 f(\theta)$ and write it in the matrix form. 

- 3.1 (1) If $Ker X = 0$, is it true that $\nabla^2 f(\theta) > 0$ ?


# Solution
here

In [ ]:
# and here
def hessf(theta: np.ndarray):
    pass

# Problem 4. (1.5) Newton's method
Another classical optimization method is the Newton's method:
$$\theta^{k+1} = \theta^{k} - [\nabla^2 f(\theta^k)]^{-1} \nabla f(\theta^k)$$

- Implement the method. 

- Describe its behaviour on our optimization problem. Try different starting points $\theta^0$.



In [ ]:
# solution here

and here

# Problem 5. (1) Damped Newton's method 

Damped Newton's method is a modification of the original one with introduced step-sizes $h^k > 0$:
$$\theta^{k+1} = \theta^{k} - \color{red}{h^k} [\nabla^2 f(\theta^k)]^{-1} \nabla f(\theta^k)$$

Common-used strategy for choosing step-size is the Armijo rule. Armijo rule applies for the methods of the following form:
$$\theta^{k+1} = \theta^k - h^k g^k.$$
Armijo rule (backtracking linesearch) is the following algorithm:
1. Set $h^k = h_0$
2. While $f(\theta^k - h^k g^k) > f(\theta^k) - \alpha h^k (g^k)^\top \nabla f(\theta^k)$ do $h^k = h^k \cdot \rho$.

Here $\alpha \geq 0$ and $0 < \rho < 1$ are hyperparameters. Usually $\alpha \leq 0.3$.

- Show that Armijo rule is correctly defined for damped Newton's method applied to smooth $f$, i.e. while-loop terminates with some $h^k > 0$

- Implement damped Newton's method. How much worse (roughly) arithmetic complexity of its iteration comparing to classical Newton's method?

# Solution
here

In [ ]:
# and here

# Problem 6*. (3) Speedup

- Suggest (it's possible you've done it already in problems 4-5) the way to recalculate $g^k$ with $O(n^2)$  arithmetic complexity (matrix $X \in \mathbb R^{O(n) \times O(n)}$), meaning that the first iteration can have the complexity up to $O(n^3)$, and all the rest are $O(n^2)$.
- Implement it

# Solution
here

In [ ]:
# and here

# Problem 7 (2) (offtop about GD). 

Упр 1.2 пособия: покажите, что если для минимизации положительно определённой квадратичной формы $f(x)=\frac{1}{2}\langle A x, x\rangle-\langle b, x\rangle \rightarrow \min _{x \in \mathbb{R}^n}$ использовать градиентный спуск с $h^k=1 / \lambda_{k+1}$ где $\lambda_{k+1}-(k+1)$-е собственное значение матрицы $A\left(0<\mu=\lambda_1 \leqslant \ldots \leqslant \lambda_n=L\right)$, то независимо от точки старта метод будет конечен: $x^n=x_*$, где $A x_*=b$. 


# Solution
here 

# Problem 8 (2) (offtop about GD 2)

Покажите, что в случае вырожденной квадратичной функции $f(x) = \frac12 \langle Ax, x\rangle - \langle b, x\rangle, \; \mu = 0$ градиентный спуск линейно сходится ко множеству $\{x: Ax - b \in \text{Ker } A\}$. Сходится ли GD на такой задаче в обычном смысле этого слова? Вспомните о том, что когда функция не сильно выпукла на всей области определения, она всё ещё может быть сильно выпукла на некотором подмножестве/подпространстве.

# Solution 
here